# Importamos librerias y bajamos los archivos

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# 34. ¿Quién es el usuario que más ha bloqueado a otros? (1)

In [ ]:
df = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Data/logs.csv')
contents_rdd = df.rdd

In [ ]:
blockeadores = contents_rdd.filter(lambda x: x.logtype == "block")

In [ ]:
blockeadores.map(lambda x: (x.contributor_username, 1)).reduceByKey(lambda x,y: x+y)\
.reduce(lambda x,y: x if x[1] > y[1] else y)

('Magister Mathematicae', 30148)

# 14. El Top 5 de contenidos que tienen la mayor cantidad de redirecciones que apuntan a ellos. (1)

In [ ]:
df = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Data/redirect_list.csv')
redirect_rdd = df.rdd

In [ ]:
redirect_rdd.map(lambda x: (x.rd_title, 1)).reduceByKey(lambda x,y: x+y)\
.takeOrdered(5, lambda x: -x[1])

[('Artículo_futuro', 1161),
 ('IV_milenio', 1001),
 ('V_milenio', 998),
 ('Estaciones_de_Metrobús_de_la_Ciudad_de_México', 396),
 ('163.117.0.0', 310)]

# 17. Para aquel contenido georeferenciado publicado anónimamente indicar por país, cuántas IPs de usuarios corresponden a IPv4 y cuantas a IPv6. (1)

*   IPv4: 179.53.218.243
*   IPv6: 2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2.




In [ ]:
df = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Data/geo_tags.csv')
geo_rdd = df.rdd

In [ ]:
df = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Data/contents.csv')
contents_rdd = df.rdd

In [ ]:
rddById_ip = contents_rdd.filter(lambda x: x.revisor_ip != None).map(lambda x: (x.id, x.revisor_ip)).cache()

In [ ]:
rddById_country = geo_rdd.filter(lambda x: x.gt_country != None).map(lambda x: (x.gt_page_id, x.gt_country))

In [ ]:
rddById_country.join(rddById_ip).map(lambda x: (x[1],((x[1][0],'IPv6') if ':' in x[1][1] else (x[1][0],'IPv4'))))\
.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y)\
.sortBy(lambda x: x[0][0]).collect()

[(('AE', 'IPv4'), 1),
 (('AE', 'IPv6'), 1),
 (('AF', 'IPv4'), 1),
 (('AQ', 'IPv4'), 5),
 (('AQ', 'IPv6'), 1),
 (('AR', 'IPv6'), 6),
 (('AR', 'IPv4'), 51),
 (('AT', 'IPv4'), 8),
 (('AU', 'IPv4'), 4),
 (('BE', 'IPv4'), 6),
 (('BO', 'IPv6'), 11),
 (('BO', 'IPv4'), 10),
 (('BR', 'IPv6'), 1),
 (('BR', 'IPv4'), 3),
 (('BS', 'IPv4'), 2),
 (('BZ', 'IPv4'), 1),
 (('CA', 'IPv4'), 9),
 (('CF', 'IPv4'), 1),
 (('CH', 'IPv4'), 2),
 (('CH', 'IPv6'), 1),
 (('CL', 'IPv4'), 52),
 (('CL', 'IPv6'), 3),
 (('CN', 'IPv4'), 3),
 (('CN', 'IPv6'), 1),
 (('CO', 'IPv4'), 24),
 (('CR', 'IPv4'), 41),
 (('CU', 'IPv4'), 2),
 (('CZ', 'IPv4'), 1),
 (('DE', 'IPv4'), 20),
 (('DE', 'IPv6'), 3),
 (('DK', 'IPv6'), 1),
 (('DK', 'IPv4'), 2),
 (('DO', 'IPv4'), 2),
 (('DZ', 'IPv6'), 1),
 (('EG', 'IPv4'), 2),
 (('EH', 'IPv4'), 1),
 (('ES', 'IPv6'), 4),
 (('ES', 'IPv4'), 557),
 (('FI', 'IPv6'), 1),
 (('FI', 'IPv4'), 2),
 (('FR', 'IPv4'), 35),
 (('FR', 'IPv6'), 3),
 (('GB', 'IPv6'), 3),
 (('GB', 'IPv4'), 39),
 (('GI', 'IPv4'), 1),

# 15. Listado en orden de importancia (del más hablado al menos hablado) de los idiomas que manejan aquellos usuarios que hablan por lo menos tres idiomas. (2)






In [ ]:
df = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Data/languages.csv')
languajes_rdd = df.rdd

In [ ]:
languajes_3 = languajes_rdd.map(lambda x: (x.babel_user, 1)).reduceByKey(lambda x,y: x+y)\
.filter(lambda x: x[1] >=3)

In [ ]:
languajes_3.join(languajes_rdd).map(lambda x: (x[1][1],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1]).collect()

[('en', 6401),
 ('es', 6385),
 ('fr', 3488),
 ('it', 1766),
 ('de', 1710),
 ('pt', 1611),
 ('ca', 1256),
 ('ja', 483),
 ('ru', 469),
 ('gl', 464),
 ('la', 448),
 ('eo', 263),
 ('eu', 244),
 ('zh', 214),
 ('ar', 190),
 ('ast', 154),
 ('pl', 143),
 ('nl', 142),
 ('el', 125),
 ('sv', 117),
 ('he', 92),
 ('grc', 71),
 ('ko', 69),
 ('uk', 68),
 ('ro', 65),
 ('an', 59),
 ('fi', 56),
 ('no', 49),
 ('oc', 49),
 ('nah', 48),
 ('cs', 47),
 ('qu', 45),
 ('gn', 45),
 ('da', 42),
 ('hr', 42),
 ('tr', 42),
 ('lad', 42),
 ('hu', 40),
 ('sr', 38),
 ('fa', 33),
 ('bg', 31),
 ('sk', 27),
 ('be', 25),
 ('ia', 25),
 ('sl', 21),
 ('pt-BR', 21),
 ('id', 21),
 ('ext', 19),
 ('ga', 19),
 ('hi', 19),
 ('th', 18),
 ('arn', 18),
 ('et', 17),
 ('lt', 16),
 ('sh', 15),
 ('mk', 15),
 ('is', 14),
 ('io', 14),
 ('sa', 14),
 ('gsw', 14),
 ('qya', 13),
 ('bs', 13),
 ('simple', 12),
 ('scn', 12),
 ('sw', 12),
 ('nn', 12),
 ('yue', 12),
 ('af', 11),
 ('lv', 11),
 ('vec', 11),
 ('tl', 10),
 ('vi', 10),
 ('co', 10),
 ('sc'

# 29. Para los contenidos visibles en wikipedia, cuales son los artículos que tienen la máxima y mínima distancia entre ids de su revisión actual y la anterior (2)



In [ ]:
df = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Data/contents.csv')
contents_rdd = df.rdd

In [ ]:
contents_rdd = contents_rdd.filter(lambda x: x.id != None and x.id.isnumeric() and x.revision_id !=None and x.revision_id.isnumeric()and x.parent_revision != None and x.parent_revision.isnumeric())

In [ ]:
contents_rdd.reduce(lambda x,y: x if (float(x.revision_id) - float(x.parent_revision))**2 > (float(y.revision_id) - float(y.parent_revision))**2 else y)

Row(title='CEDA', id='30623', namespace='0', revision_id='138003942', parent_revision='857196', revision_timestamp='2021-08-30T10:45:27Z', revisor_username='PabloTecEspaña', revisor_id='5783560', revisor_ip=None, revisor_comment='Traducción de código.')

In [ ]:
contents_rdd.reduce(lambda x,y: x if (float(x.revision_id) - float(x.parent_revision))**2 < (float(y.revision_id) - float(y.parent_revision))**2 else y)

Row(title='Río Mātukituki', id='10010546', namespace='0', revision_id='138051023', parent_revision='138051022', revision_timestamp='2021-09-01T10:25:09Z', revisor_username='Lorito987', revisor_id='5952489', revisor_ip=None, revisor_comment=None)

# 25. Obtenga con spark los datos (de forma ya agregada) que le permitan realizar la siguiente visualización y realice la misma (3)

In [ ]:
df = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Data/logs.csv')
logs_rdd = df.rdd

In [ ]:
sample_logs_rdd = logs_rdd.sample(False, 0.01)

In [ ]:
sample_logs_rdd = sample_logs_rdd.filter(lambda x: x.logtype in ['autocreate','block','create','delete','move']and x.contributor_username != None)\
.map(lambda x: (x.contributor_username, x.logtype)).cache()

In [ ]:
logs_by_people = sample_logs_rdd.map(lambda x: (x[0],x[1]))

In [ ]:
logs_by_people.take(10)

[('Sanbec', 'delete'),
 ('Joseaperez', 'delete'),
 ('Sabbut', 'delete'),
 ('Sabbut', 'delete'),
 ('Dodo', 'delete'),
 ('Dodo', 'delete'),
 ('Dodo', 'delete'),
 ('Cookie', 'delete'),
 ('Dodo', 'delete'),
 ('JorgeGG', 'delete')]

In [ ]:
logs_by_people.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y).take(3)

[('block', 3582), ('create', 11983), ('move', 9512)]

In [ ]:
logs_by_people.groupByKey().mapValues(list).take(3)

[('Joseaperez',
  ['delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'block',
   'move',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete']),
 ('Javier Carro',
  ['delete',
   'delete',
   'delete',
   'move',
   'move',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete']),
 ('Wricardoh',
  ['delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'delete',
   'd